## CAGE raw energy spectrum checker

This notebook is intended to complement `energy_cal.py`.  
We use the interactive mode to load a raw spectrum from a particular set of cycle files, and use it to pick out the raw locations of the peaks, which can then be added to `metadata/input_peaks.json` as input guesses.

Run this notebook using the `legend-base` Shifter image.  [Here are the instructions to set this up.](https://github.com/legend-exp/legend/wiki/Computing-Resources-at-NERSC)

In [2]:
# install user prerequisites
# !pip install ipympl==0.5.7 --user

# Use this at NERSC to get interactive plots.
%matplotlib widget

import os, h5py
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from pygama import DataGroup, lh5
import pygama.analysis.histograms as pgh

#### >>> Users, set config here ! <<<
Set the query here to DataGroup to load files.  You may want to refer to `runDB.json` to see how to vary this.  Here we also set the energy parameter of interest. 

In [3]:
# fileDB query
# que = 'run==66 and cycle > 885'
que = 'run==280'

# energy estimator of interest
# etype = 'energy'
etype = 'trapEftp'

# lh5 table name
tb_in = 'ORSIS3302DecoderForEnergy/dsp'

# uncalibrated energy range
# xlo, xhi, xpb = 0, 4e6, 10000   # good for energy
xlo, xhi, xpb = 0, 10000, 10  # good for trapEmax and trapEftp

# load the fileDB and make sure the entries exist
dg = DataGroup('$CAGE_SW/processing/cage.json', load=True)
dg.fileDB.query(que, inplace=True)
if len(dg.fileDB)==0:
    print('Error, no files found.  Check your query, and fileDB.h5.')

ecal_cols = ['run', 'cycle', 'skip', 'runtype', 'startTime', 'threshold', 'stopTime', 'runtime']
dg.fileDB[ecal_cols]

,run,cycle,skip,runtype,startTime,threshold,stopTime,runtime
2183,280,2185,False,alp,1.626918e+09,16.0,1.626920e+09,29.988143
2184,280,2186,False,alp,1.626920e+09,16.0,1.626922e+09,29.974719
2185,280,2187,False,alp,1.626922e+09,16.0,1.626924e+09,29.996447
2186,280,2188,False,alp,1.626924e+09,16.0,1.626926e+09,29.973354
2187,280,2189,False,alp,1.626926e+09,16.0,1.626927e+09,29.979839
2188,280,2190,False,alp,1.626927e+09,16.0,1.626929e+09,29.987426
2189,280,2191,False,alp,1.626929e+09,16.0,1.626931e+09,29.974210
2190,280,2192,False,alp,1.626931e+09,16.0,1.626933e+09,29.974028
2191,280,2193,False,alp,1.626933e+09,16.0,1.626935e+09,29.998159
2192,280,2194,False,alp,1.626935e+09,16.0,1.626936e+09,30.005218


#### Load data
Here we use DataGroup's fileDB to select files, retrieve DSP data,
and show some information about what we've selected.

In [5]:
# essentially the same code as in energy_cal::check_raw_spectrum

# load numpy arrays of uncalibrated energy
dsp_list = dg.lh5_dir + dg.fileDB['dsp_path'] + '/' + dg.fileDB['dsp_file']
raw_data = lh5.load_nda(dsp_list, [etype], tb_in, verbose=False)

# get runtime
runtime_min = dg.fileDB['runtime'].sum()

# print columns of table
with h5py.File(dsp_list.iloc[0], 'r') as hf:
    print('\nLH5 columns:', list(hf[f'{tb_in}'].keys()))
    
# histogram energy data for this estimator and normalize by runtime
data = raw_data[etype]
hist, bins, var = pgh.get_hist(data, range=(xlo, xhi), dx=xpb)
bins = bins[1:] # trim zero bin, not needed with ds='steps'
hist_rt = np.divide(hist, runtime_min * 60)

print(f'\nRaw E: {etype}, {len(data)} cts, runtime: {runtime_min:.2f} min')


LH5 columns: ['A_10', 'AoE', 'ToE', 'atrap_max', 'bl', 'bl_int', 'bl_mean', 'bl_sig', 'bl_slope', 'bl_slope_ftp', 'channel', 'dcr', 'dtrap_max', 'energy', 'fltp', 'fltp_sig', 'hf_max', 'lf_max', 'lt_int', 'lt_mean', 'lt_sig', 'lt_slope', 'qdrift', 'timestamp', 'tp_0', 'tp_02', 'tp_05', 'tp_10', 'tp_20', 'tp_30', 'tp_40', 'tp_50', 'tp_60', 'tp_70', 'tp_80', 'tp_90', 'tp_96', 'tp_max', 'trapE_argmax', 'trapEftp', 'trapEmax', 'triE', 'wf_argmax', 'wf_max']

Raw E: trapEftp, 3188241 cts, runtime: 837.60 min


#### Create interactive spectrum

In [6]:
%matplotlib widget
plt.semilogy(bins, hist_rt, ds='steps', c='b', lw=1, label=etype)
plt.xlabel(etype, ha='right', x=1)
plt.ylabel(f'cts/sec, {xpb}/bin', ha='right', y=1)
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [ ]:
# draw a few lines in the uncalibrated spectrum

# %matplotlib widget
%matplotlib inline
plt.semilogy(bins, hist_rt, ds='steps', c='b', lw=1, label=etype)

# plt.axvline(5120, c='r', lw=1)
# plt.axvline(1665, c='b', lw=1)

plt.xlabel(etype, ha='right', x=1)
plt.ylabel(f'cts/sec, {xpb}/bin', ha='right', y=1)
plt.show()


In [ ]:
# apply an arbitrary set of calibration constants
# can be the output of np.polyfit

# pars = [-4.14621058e-06, 5.21111045e-01, 5.47450830e+01]
# pars = [ 4.79137964e-07, 9.98176163e-01, -4.24949030e-02]
pars = [0, 1, 0]
pars = [-4.28582919e-06, 5.20891424e-01, 5.47741940e+01]

xlo, xhi, xpb = 0, 500, 1
# xlo, xhi, xpb = 0, 100, 1

pfunc = np.poly1d(pars)
cal_data = pfunc(raw_data[etype])
hist, bins, var = pgh.get_hist(cal_data, range=(xlo, xhi), dx=xpb)
bins = bins[1:]
hist_rt = np.divide(hist, runtime_min * 60)

%matplotlib widget

plt.semilogy(bins, hist_rt, ds='steps', c='b', lw=1, label=etype)

plt.xlabel(etype, ha='right', x=1)
plt.ylabel(f'cts/sec, {xpb}/bin', ha='right', y=1)
plt.show()

In [ ]:
%matplotlib widget
plt.semilogy(bins, hist_rt, ds='steps', c='b', lw=1, label=etype)
plt.xlabel(etype, ha='right', x=1)
plt.ylabel(f'cts/sec, {xpb}/bin', ha='right', y=1)
plt.show()